In [ ]:
%run includes/google-audit-setup.ipynb

In [ ]:
# discover folders and projects
def get_projects_query(entity_id):
    return f"""
    SELECT displayName, name, parent, projectId 
    FROM google.cloudresourcemanager.projects 
    WHERE parent = '{entity_id}';
    """

def get_folders_query(entity_id):
    return f"""
    SELECT displayName, name, parent 
    FROM google.cloudresourcemanager.folders 
    WHERE parent = '{entity_id}';
    """

resources_df, projects, project_numbers, tree = get_all_resources(get_projects_query, get_folders_query, org_id.value)
tree

In [ ]:
# get project/regions and usage summary info

queries = [
    f"""
    SELECT name as 'region',
    '{project}' as project,
    zones,
    quotas
    FROM google.compute.regions WHERE project = '{project}'
    """
    for project in projects
]

regions_df = run_stackql_queries(queries)

regions_df = (
    explode_json_list_col(regions_df, 'quotas', 'quota')
    .loc[explode_json_list_col(regions_df, 'quotas', 'quota')['quota_usage'] != 0]
)

In [ ]:
# network usage summary report

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the list of desired metrics
desired_metrics = ['STATIC_ADDRESSES', 'IN_USE_ADDRESSES', 'INSTANCES']

# Number of rows for subplots (assuming one chart per row)
n_metrics = len(desired_metrics)

# Create subplot structure
fig = make_subplots(rows=n_metrics, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=desired_metrics)

for idx, metric in enumerate(desired_metrics, start=1):
    subset = regions_df[regions_df['quota_metric'] == metric]
    fig.add_trace(go.Bar(x=subset['project'], y=subset['quota_usage'], name=metric), row=idx, col=1)

fig.update_layout(height=200*n_metrics, width=800, title_text="Quota Usage by Project per Metric", showlegend=False)
fig.show()

In [ ]:
# get networks

queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    subnetworks
    FROM google.compute.networks WHERE project = '{project}'
    """
    for project in projects
]

networks_df = (lambda df: df[df['name'] != 'null'])(run_stackql_queries(queries))
networks_df

In [ ]:
# network peerings
networks_subset_df = networks_df[networks_df['peerings'] != 'null'][['name', 'peerings']].copy()
network_peerings_df = explode_json_list_col(networks_subset_df, 'peerings', 'peering')
network_peerings_df

In [ ]:
# get project iam bindings

queries = [
    f"""
    SELECT role, condition, members, '{project}' as project
    FROM google.cloudresourcemanager.projects_iam_policies
    WHERE projectsId = '{project}'
    """
    for project in projects
]

_bindings_df = run_stackql_queries(queries)

project_bindings = (explode_json_list_col(_bindings_df, 'members', 'member')
 .assign(member_type=lambda x: x['member'].str.split(':', expand=True)[0],
         member_email=lambda x: x['member'].str.split(':', expand=True)[1])
)
project_bindings

In [ ]:
# get all instances across all projects

queries = [
    f"""
    SELECT 
    name,
    status,
    SPLIT_PART(machineType, '/', 11) as machineType,
    '{project}' as project,
    creationTimestamp,
    lastStartTimestamp,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].network'), '/', 10) as network,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].subnetwork'), '/', 11) as subnetwork,
    JSON_EXTRACT(networkInterfaces, '$[0].networkIP') as networkIP,
    JSON_EXTRACT(tags, '$.items') as tags,
    labels
    FROM google.compute.instances
    WHERE project = '{project}'
    """
    for project in projects
]
instances_df = run_stackql_queries(queries)	
instances_df

In [ ]:
args_json = json.dumps({"project": projects[0]})

In [ ]:
%%stackql
SELECT * FROM google.compute.regions WHERE project = '{project}'

In [ ]:
# cloud functions
queries = [
    f"""
    SELECT locationId,
    '{project}' as project
    FROM google.cloudfunctions.locations WHERE projectsId = '{project}'
    """
    for project in projects
]

locations_df = run_stackql_queries(queries)
locations = locations_df['locationId'].tolist()

queries = [
    f"""
    SELECT *,
    '{project}' as project,
    '{location}' as location
    FROM google.cloudfunctions.functions WHERE projectsId = '{project}' AND locationsId = '{location}'
    """
    for project in projects
    for location in locations
]
functions_df = run_stackql_queries(queries)
functions_df